#Simulación de eventos discretos en Simpy
Por: Benjamin Cruz Álvarez

##Conceptos a evaluar
- Colas
- Recursos
- Servidores en paralelo
- Servidores en serie
- Medidas de desempeño


##Descripción del sistema
En esta práctica veremos como implementar una simulación donde utilicemos estos conceptos usando la librería SimPy.
Primero consideremos el siguiente enunciado:

Tenemos una carnicería que cuenta con dos carniceros quienes se encargan de atender a los clientes en orden de llegada, es decir, si uno de los carniceros se desocupa, atiende al que sigue en la fila, si los dos carniceros están libres el cliente es atendido por cualquiera de los dos (sin preferencia). Luego de esto, los clientes pasan a la caja a pagar su pedido. El sistema se ve representado así:

![Sistema](https://i.ibb.co/QrnnkY4/fff.png)

Podemos apreciar que los clientes (entidades) que llegan al sistema, hacen la cola si los carniceros están ocupados, de lo contrario pasan a cualquiera de los dos que estén desocupados.

Ahora bien, ya que sabemos como funciona el sistema definamos los parámetros para determinar el tiempo entre llegadas y la duración del servicio.

- El tiempo entre llegadas se distribuye de manera exponencial con media 180 segundos.

- El tiempo de servicio del carnicero 1 y 2 (Recursos, Servidor 1), se distribuye uniforme entre 360 segundos y 540 segundos

- El tiempo de servicio de la caja (Servidor 2) se distribuye uniforme entre 180 y 360 segundos.

- El tiempo de viaje entre el servidor 1 y la cola para el servidor 2 es despreciable.

- Tiempo simulación: 1800 segundos

Cabe aclarar que estos parámetros son totalmente modificables, como se verá más adelante. Ahora pasaremos a programarlo en Python utilizando la librería SimPy.

¡Muy importante! Siempre antes de usar la librería es necesario instalarla. Recuerde configurar su tipo de entorno de ejecución a Python 3  








In [0]:
!pip install simpy

Luego de haber instalado la librería es igual de importante importarla, además importaremos algunas librerías nativas de Python que nos seran útiles para generar las llegadas y tiempos de servicio.



In [0]:
import random              #Se importa la librería para generar números pseudoaleatorios
import math                #Se importa a librería para usar funciones matemáticas como logaritmo natural
import simpy               #Se importa la librería necesaria para simular. (Es necesario instalarla)

Ahora definimos algunas constantes importantes que utilizaremos a lo largo de la simulación.




In [0]:
NroClientes = 5            #Definimos la cantidad de clientes que queremos que lleguen a nuestro sistema
TiempoSimulacion = 1800    #Si queremos podemos definir un tiempo de simulación(Ya no dependerá del número de clientes). Recordemos que está en segundos
LlegaMedia = 180           # Parámetro de la distribución exponencial
MinServ1 = 360             # Parámetros de la distribución uniforme para el tiempo de servicio en el servidor 1
MaxServ1 = 540
MinServ2 = 180             # Parámetros de la distribución uniforme para el tiempo de servicio en el servidor 1
MaxServ2 = 360
RSeed = 40                 # Usaremos como semilla para replicar la secuencia de números pseudoaleatorios
#---------Variables de desempeño: estos auxiliares nos van a servir para calcular las variables de desempeño -------#
UsoServ1 = 0.0             #Utilización del servidor 1
UsoServ2 = 0.0             #Utilización del servidor 2
TiempoFinUltCliente = 0.0  #Tiempo total de las entidades en el sistema (Sólo toma en cuenta las que salieron del sistema)
NroClientesAtendidos = 0.0 #Cantidad de clientes que salieron del sistema 
TiempoEspCola1 = 0.0       #Tiempo total espera en cola 1 (Solo tiene en cuenta los clientes que salieron de cola)
NroClientesSalCo1 = 0.0    #Número de clientes que salieron de cola 1
TiempoEspCola2 = 0.0       #Tiempo total espera en cola 2
NroClientesSalCo2= 0.0     #Número de clientes que salieron de cola 2

Es posible modificar el tipo de distribución que siguen las llegadas y tiempos de servicio, lo único que se debe hacer es definir los parámetros de la distribución y reemplazar en las métodos que se mostrarán posteriormente, la función que genere la variable aleatoria deseada (por el método de transformada inversa, por ejemplo).

Una vez definidas estas variables, se escribe el programa principal

In [0]:
print("Inicio de la simulación")
random.seed(RSeed)
env = simpy.Environment()                 #Creamos el ambiente de simulación
serv1 = simpy.Resource(env, 2)            #Creamos el servidor 1, las entrada env representa el ambiente al que está asociado el servidor
                                          #El 2 representa el número de recursos(carniceros)
serv2 = simpy.Resource(env, 1)            # Igual que el anterior, pero notar que en este sólo hay un recurso (La caja)
env.process(main(env, serv1, serv2))      #Invocamos el método principal donde se configura la simulación.
env.run(until = TiempoSimulacion)         #Corremos la simulación. Si queremos que la simulación corra para un número de clientes deseado solo debemos quitar la instrucción 'until = TiempoSimulacion'
                                          #Además, tendremos que cambiar la función 'main', como se indicará posteriormente.

Una vez configuramos el programa principal, procedemos a definir la función 'main' que será la cual controla la simulación.

In [0]:
def main(env, serv1, serv2):                    #Recordemos los parámetros que le enviamos en el fragmento de código anterior
  i = 0
  while True:                                   #Se ejecuta hasta que se acabe el tiempo
    r = random.random()                         #Generamos número aleatorio
    llegada = -LlegaMedia*math.log(r)           #Transformada inversa para la distribución exponencial
    yield env.timeout(llegada)                  #Se deja trasncurri el tiempo de llegada, la simulación empieza en 0    
    env.process(cliente(env, i, serv1, serv2))  # Una vez transcurrido el tiempo de una llegada, se invoca la función cliente que se encarga de simular el comportamiento de esta entidad
    i += 1

Si decidimos simularpara un número de clientes deseado, debemos cambiar el código anterior por el siguiente:

In [0]:
def main(env, serv1, serv2):                           #Recordemos los parámetros que le enviamos en el fragmento de código anterior
  for i in range(NroClientes):                         #Se generan N clientes
    r = random.random()                                #Generamos número aleatorio
    llegada = -LlegaMedia*math.log(r)                  #Transformada inversa para la distribución uniforme
    yield env.timeout(llegada)                         #Se deja trasncurrir el tiempo de llegada, la simulación empieza en 0
    env.process(cliente(env, i, serv1, serv2))         #Una vez transcurrido el tiempo de una llegada, se invoca la función cliente que se encarga de simular el comportamiento de esta entidad

Ahora debemos crear la función con la cual gestionamos el comportamiento de los clientes

In [0]:
def cliente(env, nombre, serv1, serv2):           #Le pasamos los parámetros que definimos en el bloque de código anterior, en este caso el nombre del cliente es 'i' con i = 1,...,n
  global TiempoEspCola1                           #Esta línea nos permite modificar la variable TiempoEspCola1 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  global TiempoEspCola2                           #Esta línea nos permite modificar la variable TiempoEspCola2 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  global NroClientesSalCo1
  global NroClientesSalCo2
  llega = env.now                                 #registramos el tiempo en el que llegó el cliente al sistema
  print("T = %.1f -> Cliente %d llegó al sistema" % (llega, nombre + 1))
  
  with serv1.request() as req:                    #Se pregunta si el serv1 está ocupado (Alguno de los carniceros), si el servidor está ocupado el cliente se encola
    yield req                                     #El cliente obtiene el turno
    pasa = env.now                                # Se obtiene el tiempo en el que el cliente pasa a ser atendido en el servidor 1
    espera = pasa - llega                         # Se calcula el tiempo que estuvo en cola
    TiempoEspCola1 += espera
    NroClientesSalCo1 +=1
    print ("T = %.1f -> Cliente %d empieza a ser atendido por carnicero, después de %.1f segundos en cola" % (pasa, nombre + 1, espera))
    yield env.process(carnicero(nombre))          #Se invoca el proceso donde es atendido el cliente
    
    serv1.release(req)                            # Una vez el cliente es atendido por un carnicero, se libera el recurso para que pueda atender a otro cliente
    Qpagar = env.now                              #Se registra el tiempo cuando el cliente sale del servidor 1 y entra a la cola del servidor 2
    with serv2.request() as req2:                 #Se pregunta si el servidor 2 (la caja) está ocupado, en caso de estarlo se encola
      yield req2                                  #Obtiene un turno
      pagar = env.now                             #En este momento el cliente salió de la cola y pasa a pagar, se registra este tiempo para calcular la espera
      espera2 = pagar - Qpagar                    # Se calcula el tiempo que estuvo en cola
      TiempoEspCola2 += espera2
      NroClientesSalCo2 +=1
      print ("T = %.1f-> Cliente %d empieza a ser atendido en caja, después de %.1f segundos en cola" % (pagar, nombre + 1, espera2))
      yield env.process(caja(nombre))             # Se invoca la función donde el cliente paga
      serv2.release(req2)      

Por último, es necesario programar las funciones que nos representan nuestros dos servidores, el servidor 1, de carniceros, y el servidor 2 la caja. Empecemos con el serividor 1:

In [0]:
def carnicero(nombre):
  global UsoServ1 #Esta línea nos permite modificar la variable UsoServ1 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  r = random.random() #Generamos un número pseudoaleatorio
  DServicio = MinServ1 + (MaxServ1 - MinServ1)*r #Se genera un tiempo de servicio a partir de la transformada inversa de la distribución uniforme
  salida = env.now + DServicio
  UsoServ1 += DServicio # Acumulamos el tiempo de uso del servidor 1
  print ("T = %.1f -> Cliente %d sale del puesto de carnicero" %(salida, nombre + 1))
  yield env.timeout(DServicio) # Se deja transcurrir la duración del servicio en el sistema

Ahora configuramos el servidor dos y el programa estará listo para simular la carnicería.

In [0]:
def caja(nombre):
  global UsoServ2                                        #Esta línea nos permite modificar la variable UsoServ2 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  global NroClientesAtendidos
  global TiempoFinUltCliente
  r = random.random()                                    #Generamos un número pseudoaleatorio
  DServicio = MinServ2 + (MaxServ2 - MinServ2)*r         #Se genera un tiempo de servicio a partir de la transformada inversa de la distribución uniforme.
                                                         #note que usamos el mínimo y el máximo para el servidor 2
  salida = env.now + DServicio
  UsoServ2 += DServicio                                  #Acumulamos el tiempo de uso del servidor 2
  print ("T = %.1f -> Cliente %d paga su compra y sale del sistema" %(salida, nombre + 1))
  yield env.timeout(DServicio)                           #Se deja transcurrir la duración del servicio en el sistema
  NroClientesAtendidos += 1
  TiempoFinUltCliente = env.now

A continuación se muestra como calcular las medidas de desempeño:

In [0]:
#------------Medidas de Desempeño---------------#
print("\n\n-------------------Medidas de desempeño-----------------------")
PorcentajeUso = (UsoServ1/2)*100/TiempoSimulacion                                                                     # Se toman los dos servidores como uno solo(se promedia su tiempo de uso) y se saca su factor porcentual respecto a el tiempo final de la simulación
print("Utilización de los puestos de carnicero: %.2f%%" % PorcentajeUso)

PorcentajeUso2 = (UsoServ2)*100/TiempoSimulacion                                                                      # Se hace de igual manera que el anterior, la diferencia es que este no se divide por dos ya que este servidor solo tiene un recurso
print("Utilización del servidor 2: %.2f%%" % PorcentajeUso2)

if (NroClientesSalCo1 == 0):
  print("Tiempo de espera promedio en cola 1 : 0 segundos")
else:
  print("Tiempo de espera promedio en cola 1 : %.2f segundos" % (TiempoEspCola1/NroClientesSalCo1))                   # Se obtiene este indicador, a partir del tiempo que esperaron las entidades que lograron salir de la cola


if (NroClientesSalCo2 == 0):
  print("Tiempo de espera promedio en cola 2 : 0 segundos")
else:
   print("Tiempo de espera promedio en cola 2 : %.2f segundos" % (TiempoEspCola2/NroClientesSalCo2))                  # De igual manera que la anterior pero para la cola del servidor 2
    
if (NroClientesAtendidos == 0):
  print("Tiempo de promedio de clientes en el sistema : 0 segundos")
else:
  print("Tiempo de promedio de clientes en el sistema : %.2f segundos" % (TiempoFinUltCliente/NroClientesAtendidos))    # Se obtiene a partir del tiempo que registraron en el sistema las entidades que lograron salir del sistema.

Así quedaría el código completo:

In [35]:
import random              #Se importa la librería para generar números pseudoaleatorios
import math                #Se importa a librería para usar funciones matemáticas como logaritmo natural
import simpy               #Se importa la librería necesaria para simular. (Es necesario instalarla)

NroClientes = 5            #Definimos la cantidad de clientes que queremos que lleguen a nuestro sistema
TiempoSimulacion = 1800    #Si queremos podemos definir un tiempo de simulación(Ya no dependerá del número de clientes). Recordemos que está en segundos
LlegaMedia = 180           # Parámetro de la distribución exponencial
MinServ1 = 360             # Parámetros de la distribución uniforme para el tiempo de servicio en el servidor 1
MaxServ1 = 540
MinServ2 = 180             # Parámetros de la distribución uniforme para el tiempo de servicio en el servidor 1
MaxServ2 = 360
RSeed = 40                 # Usaremos como semilla para replicar la secuencia de números pseudoaleatorios
#---------Variables de desempeño: estos auxiliares nos van a servir para calcular las variables de desempeño -------#
UsoServ1 = 0.0             #Utilización del servidor 1
UsoServ2 = 0.0             #Utilización del servidor 2
TiempoFinUltCliente = 0.0  #Tiempo total de las entidades en el sistema (Sólo toma en cuenta las que salieron del sistema)
NroClientesAtendidos = 0.0 #Cantidad de clientes que salieron del sistema 
TiempoEspCola1 = 0.0       #Tiempo total espera en cola 1 (Solo tiene en cuenta los clientes que salieron de cola)
NroClientesSalCo1 = 0.0    #Número de clientes que salieron de cola 1
TiempoEspCola2 = 0.0       #Tiempo total espera en cola 2
NroClientesSalCo2= 0.0     #Número de clientes que salieron de cola 2



def main(env, serv1, serv2):                    #Recordemos los parámetros que le enviamos en el fragmento de código anterior
  i = 0
  while True:                                   #Se ejecuta hasta que se acabe el tiempo
    r = random.random()                         #Generamos número aleatorio
    llegada = -LlegaMedia*math.log(r)           #Transformada inversa para la distribución exponencial
    yield env.timeout(llegada)                  #Se deja trasncurri el tiempo de llegada, la simulación empieza en 0    
    env.process(cliente(env, i, serv1, serv2))  # Una vez transcurrido el tiempo de una llegada, se invoca la función cliente que se encarga de simular el comportamiento de esta entidad
    i += 1
    

    
def cliente(env, nombre, serv1, serv2):           #Le pasamos los parámetros que definimos en el bloque de código anterior, en este caso el nombre del cliente es 'i' con i = 1,...,n
  global TiempoEspCola1                           #Esta línea nos permite modificar la variable TiempoEspCola1 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  global TiempoEspCola2                           #Esta línea nos permite modificar la variable TiempoEspCola2 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  global NroClientesSalCo1
  global NroClientesSalCo2
  llega = env.now                                 #registramos el tiempo en el que llegó el cliente al sistema
  print("T = %.1f -> Cliente %d llegó al sistema" % (llega, nombre + 1))
  
  with serv1.request() as req:                    #Se pregunta si el serv1 está ocupado (Alguno de los carniceros), si el servidor está ocupado el cliente se encola
    yield req                                     #El cliente obtiene el turno
    pasa = env.now                                # Se obtiene el tiempo en el que el cliente pasa a ser atendido en el servidor 1
    espera = pasa - llega                         # Se calcula el tiempo que estuvo en cola
    TiempoEspCola1 += espera
    NroClientesSalCo1 +=1
    print ("T = %.1f -> Cliente %d empieza a ser atendido por carnicero, después de %.1f segundos en cola" % (pasa, nombre + 1, espera))
    yield env.process(carnicero(nombre))          #Se invoca el proceso donde es atendido el cliente
    
    serv1.release(req)                            # Una vez el cliente es atendido por un carnicero, se libera el recurso para que pueda atender a otro cliente
    Qpagar = env.now                              #Se registra el tiempo cuando el cliente sale del servidor 1 y entra a la cola del servidor 2
    with serv2.request() as req2:                 #Se pregunta si el servidor 2 (la caja) está ocupado, en caso de estarlo se encola
      yield req2                                  #Obtiene un turno
      pagar = env.now                             #En este momento el cliente salió de la cola y pasa a pagar, se registra este tiempo para calcular la espera
      espera2 = pagar - Qpagar                    # Se calcula el tiempo que estuvo en cola
      TiempoEspCola2 += espera2
      NroClientesSalCo2 +=1
      print ("T = %.1f-> Cliente %d empieza a ser atendido en caja, después de %.1f segundos en cola" % (pagar, nombre + 1, espera2))
      yield env.process(caja(nombre))             # Se invoca la función donde el cliente paga
      serv2.release(req2)       
      
      
  
def carnicero(nombre):
  global UsoServ1 #Esta línea nos permite modificar la variable UsoServ1 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  r = random.random() #Generamos un número pseudoaleatorio
  DServicio = MinServ1 + (MaxServ1 - MinServ1)*r #Se genera un tiempo de servicio a partir de la transformada inversa de la distribución uniforme
  salida = env.now + DServicio
  UsoServ1 += DServicio # Acumulamos el tiempo de uso del servidor 1
  print ("T = %.1f -> Cliente %d sale del puesto de carnicero" %(salida, nombre + 1))
  yield env.timeout(DServicio) # Se deja transcurrir la duración del servicio en el sistema
  
  
def caja(nombre):
  global UsoServ2                                        #Esta línea nos permite modificar la variable UsoServ2 y que los cambios se conserven en todo el código, si no se pone, la variable solo se cambia localmente
  global NroClientesAtendidos
  global TiempoFinUltCliente
  r = random.random()                                    #Generamos un número pseudoaleatorio
  DServicio = MinServ2 + (MaxServ2 - MinServ2)*r         #Se genera un tiempo de servicio a partir de la transformada inversa de la distribución uniforme.
                                                         #note que usamos el mínimo y el máximo para el servidor 2
  salida = env.now + DServicio
  UsoServ2 += DServicio                                  #Acumulamos el tiempo de uso del servidor 2
  print ("T = %.1f -> Cliente %d paga su compra y sale del sistema" %(salida, nombre + 1))
  yield env.timeout(DServicio)                           #Se deja transcurrir la duración del servicio en el sistema
  NroClientesAtendidos += 1
  TiempoFinUltCliente = env.now
  
  
  
  
print("Inicio de la simulación")
#random.seed(RSeed)
env = simpy.Environment()                 #Creamos el ambiente de simulación
serv1 = simpy.Resource(env, 2)            #Creamos el servidor 1, las entrada env representa el ambiente al que está asociado el servidor
                                          #El 2 representa el número de recursos(carniceros)
serv2 = simpy.Resource(env, 1)            # Igual que el anterior, pero notar que en este sólo hay un recurso (La caja)
env.process(main(env, serv1, serv2))      #Invocamos el método principal donde se configura la simulación.
env.run(until = TiempoSimulacion)         #Corremos la simulación. Si queremos que la simulación corra para un número de clientes deseado solo debemos quitar la instrucción 'until = TiempoSimulacion'
                                          #Además, tendremos que cambiar la función 'main', como se indicará posteriormente.

#------------Medidas de Desempeño---------------#
print("\n\n-------------------Medidas de desempeño-----------------------")
PorcentajeUso = (UsoServ1/2)*100/TiempoSimulacion                                                                     # Se toman los dos servidores como uno solo(se promedia su tiempo de uso) y se saca su factor porcentual respecto a el tiempo final de la simulación
print("Utilización de los puestos de carnicero: %.2f%%" % PorcentajeUso)

PorcentajeUso2 = (UsoServ2)*100/TiempoSimulacion                                                                      # Se hace de igual manera que el anterior, la diferencia es que este no se divide por dos ya que este servidor solo tiene un recurso
print("Utilización del servidor 2: %.2f%%" % PorcentajeUso2)

if (NroClientesSalCo1 == 0):
  print("Tiempo de espera promedio en cola 1 : 0 segundos")
else:
  print("Tiempo de espera promedio en cola 1 : %.2f segundos" % (TiempoEspCola1/NroClientesSalCo1))                   # Se obtiene este indicador, a partir del tiempo que esperaron las entidades que lograron salir de la cola


if (NroClientesSalCo2 == 0):
  print("Tiempo de espera promedio en cola 2 : 0 segundos")
else:
   print("Tiempo de espera promedio en cola 2 : %.2f segundos" % (TiempoEspCola2/NroClientesSalCo2))                  # De igual manera que la anterior pero para la cola del servidor 2
    
if (NroClientesAtendidos == 0):
  print("Tiempo de promedio de clientes en el sistema : 0 segundos")
else:
  print("Tiempo de promedio de clientes en el sistema : %.2f segundos" % (TiempoFinUltCliente/NroClientesAtendidos))    # Se obtiene a partir del tiempo que registraron en el sistema las entidades que lograron salir del sistema.

Inicio de la simulación
T = 263.6 -> Cliente 1 llegó al sistema
T = 263.6 -> Cliente 1 empieza a ser atendido por carnicero, después de 0.0 segundos en cola
T = 692.9 -> Cliente 1 sale del puesto de carnicero
T = 393.4 -> Cliente 2 llegó al sistema
T = 393.4 -> Cliente 2 empieza a ser atendido por carnicero, después de 0.0 segundos en cola
T = 886.3 -> Cliente 2 sale del puesto de carnicero
T = 562.1 -> Cliente 3 llegó al sistema
T = 655.7 -> Cliente 4 llegó al sistema
T = 692.9-> Cliente 1 empieza a ser atendido en caja, después de 0.0 segundos en cola
T = 898.9 -> Cliente 1 paga su compra y sale del sistema
T = 692.9 -> Cliente 3 empieza a ser atendido por carnicero, después de 130.8 segundos en cola
T = 1165.8 -> Cliente 3 sale del puesto de carnicero
T = 886.3 -> Cliente 4 empieza a ser atendido por carnicero, después de 230.6 segundos en cola
T = 1303.6 -> Cliente 4 sale del puesto de carnicero
T = 898.9-> Cliente 2 empieza a ser atendido en caja, después de 12.6 segundos en cola


##Referencias
- García (2018). *Simulación en Python usando Simpy: llegadas y servicio*. Disponible en https://naps.com.mx/blog/simulacion-en-python-usando-simpy/

- Team SimPy (2019). *simpy.resources — Shared resource primitives* .Disponible en https://simpy.readthedocs.io/en/latest/api_reference/simpy.resources.html#module-simpy.resources.resource